In [147]:
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict, Counter
import random

In [148]:
def fix_unicode(text):
    return text.replace(u"\u2019","")

In [149]:
url = "https://www.oreilly.com/radar/rift-between-junior-and-senior-developers/"
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
content = soup.find("div", "post-radar-content") # encontra conteúdo de entrada div
regex = r"[\w']+|[\.]"

document = []
for paragraph in content.find_all('p'):
    words = re.findall(regex, fix_unicode(paragraph.text))
    document.extend(words)


print(document)

['Im', 'worried', 'about', 'AI', '.', 'Im', 'not', 'worried', 'about', 'it', 'taking', 'my', 'job', '.', 'I', 'believe', 'AI', 'is', 'a', 'genuine', 'productivity', 'tool', '.', 'By', 'which', 'I', 'mean', 'it', 'can', 'make', 'developers', 'produce', 'more', '.', 'Join', 'the', "O'Reilly", 'online', 'learning', 'platform', '.', 'Get', 'a', 'free', 'trial', 'today', 'and', 'find', 'answers', 'on', 'the', 'fly', 'or', 'master', 'something', 'new', 'and', 'useful', '.', 'The', 'question', 'is', 'whether', 'those', 'developers', 'are', 'producing', 'something', 'good', 'or', 'not', '.', 'The', 'difference', 'between', 'an', 'experienced', 'developer', 'and', 'a', 'junior', 'is', 'that', 'an', 'experienced', 'developer', 'knows', 'The', 'way', 'we', 'train', 'juniors', 'whether', 'its', 'at', 'university', 'or', 'in', 'a', 'boot', 'camp', 'or', 'whether', 'they', 'train', 'themselves', 'from', 'the', 'materials', 'we', 'make', 'available', 'to', 'them', 'Long', 'Live', 'the', 'Internet', '

### Bigrams

In [150]:
bigrams = zip(document,document[1:])
transitions = defaultdict(list)

for prev,current in bigrams:
    transitions[prev].append(current)

In [151]:
def generate_using_bigrams():
    current = '.'
    result = [ ]
    while True:
        next_word_candidates = transitions[current]
        current = random.choice(next_word_candidates)
        result.append(current)
        if current == '.': return " ".join(result)


generate_using_bigrams()

'With confidence .'

### Trigrams

In [152]:
trigrams = zip(document, document[1:], document[2:])

trigram_transitions = defaultdict(list)
starts = []

for prev, current, next in trigrams:
    if prev == ".":
        starts.append(current)

    trigram_transitions[(prev,current)].append(next)

print(trigram_transitions)

defaultdict(<class 'list'>, {('Im', 'worried'): ['about', 'about', 'that'], ('worried', 'about'): ['AI', 'it', 'AI', 'with'], ('about', 'AI'): ['.', '.'], ('AI', '.'): ['Im', 'Our', 'Im'], ('.', 'Im'): ['not', 'worried'], ('Im', 'not'): ['worried'], ('not', 'worried'): ['about'], ('about', 'it'): ['taking'], ('it', 'taking'): ['my'], ('taking', 'my'): ['job'], ('my', 'job'): ['.'], ('job', '.'): ['I'], ('.', 'I'): ['believe', 'want', 'do', 'still', 'dont', 'did'], ('I', 'believe'): ['AI'], ('believe', 'AI'): ['is'], ('AI', 'is'): ['a', 'not'], ('is', 'a'): ['genuine', 'team'], ('a', 'genuine'): ['productivity'], ('genuine', 'productivity'): ['tool'], ('productivity', 'tool'): ['.'], ('tool', '.'): ['By'], ('.', 'By'): ['which'], ('By', 'which'): ['I'], ('which', 'I'): ['mean'], ('I', 'mean'): ['it'], ('mean', 'it'): ['can'], ('it', 'can'): ['make'], ('can', 'make'): ['developers'], ('make', 'developers'): ['produce'], ('developers', 'produce'): ['more'], ('produce', 'more'): ['.'], ('m

In [153]:
def generate_using_trigrams():
    current = random.choice(starts)
    prev = "."
    result = [current]
    while True:
        next_word_candidates = trigram_transitions[(prev,current)]
        next_word = random.choice(next_word_candidates)
        prev,current = current,next_word
        result.append(current)
        if current == '.': return " ".join(result)


generate_using_trigrams()

'Weve been doing this for decades .'

In [154]:
grammar = {
"_S" : ["_NP _VP"],
"_NP" : ["_N",
"_A _NP _P _A _N"],
"_VP" : ["_V",
"_V _NP"],
"_N" : ["data science", "Python", "regression"],
"_A" : ["big", "linear", "logistic"],
"_P" : ["about", "near"],
"_V" : ["learns", "trains", "tests", "is"]
}

In [155]:
def is_terminal(token):
    return token[0] != "_"

def expand(grammar, tokens):
    for i, token in enumerate(tokens):
        if is_terminal(token): continue

        replacement = random.choice(grammar[token])

        if is_terminal(replacement):
            tokens[i] = replacement
        else: 
            tokens = tokens[:i] + replacement.split() + tokens[(i+1):]
            return expand(grammar,tokens)
        
    return tokens

def generate_sentence(grammar):
    return expand(grammar, ["_S"])

generate_sentence(grammar)

['big',
 'logistic',
 'linear',
 'Python',
 'about',
 'logistic',
 'regression',
 'near',
 'logistic',
 'data science',
 'about',
 'logistic',
 'Python',
 'is',
 'big',
 'regression',
 'about',
 'linear',
 'data science']

### Modelagem de Tópicos

In [156]:
def sample_from(weights):
    total = sum(weights)
    rnd = random.random()*total

    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0: return i


for i in range(100):
    print(sample_from[1,1,3])

In [157]:
documents = [
["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
["R", "Python", "statistics", "regression", "probability"],
["machine learning", "regression", "decision trees", "libsvm"],
["Python", "R", "Java", "C++", "Haskell", "programming languages"],
["statistics", "probability", "mathematics", "theory"],
["machine learning", "scikit-learn", "Mahout", "neural networks"],
["neural networks", "deep learning", "Big Data", "artificial intelligence"],
["Hadoop", "Java", "MapReduce", "Big Data"],
["statistics", "R", "statsmodels"],
["C++", "deep learning", "artificial intelligence", "probability"],
["pandas", "R", "Python"],
["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
["libsvm", "regression", "support vector machines"]
]

In [158]:
K = 4
document_topic_counts = [Counter() for _ in documents]
topic_words_counts = [Counter() for _ in range(K)]
topic_counts = [0 for _ in range(K)]
document_lengths = map(len, documents)
distinct_words = set(word for document in documents for word in document)
W = len(distinct_words)
D = len(documents)

In [162]:
def p_topic_given_document(topic, d, alpha=0.1):
    return ((document_topic_counts[d][topic] + alpha)/(document_lengths[d] + K*alpha))

def p_word_given_topic(word, topic,beta=0.1):
    return ((topic_words_counts[topic][word] + beta)/(topic_counts[topic] + W*beta))

def topic_weight(d, word, k):
    return p_word_given_topic(word,k)*p_topic_given_document(k,d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word,k) for k in range(K)])

In [ ]:
random.seed(0)

document_topics = [[random.randrange(K) for word in document] for document in documents]

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_words_counts[topic][word] += 1
        topic_counts[topic] += 1